In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [10]:
dataSet = pd.read_csv('DelhiAQI.csv')

In [3]:
print(dataSet.head())
print(dataSet.shape)

    PM2.5    PM10      NO    NO2     NOx    NH3     CO   SO2     O3  Benzene  \
0  454.58  935.18   81.52  41.78  187.66  27.54   9.29  3.41  54.94    25.24   
1  440.44  935.18   70.80  43.46  176.83  27.72  13.28  3.88  50.53    23.10   
2  409.09  935.18  132.46  41.19  141.02  28.94  29.67  2.83  19.33    19.04   
3  436.12  935.18   84.78  39.55  102.84  29.30  21.76  4.33  20.08    13.99   
4  415.88  976.99   60.24  37.41   80.12  30.84  26.19  6.17  16.00    11.14   

   Toluene  Xylene  AQI  
0    58.57   13.80  653  
1    49.37   15.63  645  
2    38.94   17.18  532  
3    27.53   16.82  561  
4    21.99   14.29  567  
(15000, 13)


In [11]:
# normalizing the data
dataSetNorm1 = dataSet.dropna()
for i in dataSetNorm1.columns:
    if i != 'AQI':
        # use the median and std of the training data to normalize (Z-score scaling)
        dataSetNorm1[i] = (dataSetNorm1[i] - dataSetNorm1[i].mean()) / dataSetNorm1[i].std()

X = dataSetNorm1.drop('AQI', axis=1)
y = dataSetNorm1['AQI']

In [12]:
# Implement deep multilayer perceptron neural network Model-I : Add a fully connected layer with 32 neurons with sigmoid activation
# and glorot uniform kernel initializer. Add a fully connected layer layer with 16
# neurons, relu activation and he uniform as kernel initializer. Add a fully
# connected layer with 1 neuron, relu activation function and he uniform as
# kernel initializer. Use Adam optimizer with batch size 16, learning rate 0.01
# and epochs set to 20.
# Test using root mean squared error as loss function.

def model1(X_train, X_val, y_train, y_val):
    model = Sequential()
    model.add(Dense(32, activation='sigmoid', kernel_initializer='glorot_uniform'))
    model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='relu', kernel_initializer='he_uniform'))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error', batch_size=16, epochs=20)

    model.fit(X_train, y_train,  validation_data=(X_val, y_val))
    
    return model

count = 5
avg_mse = 0
for i in range(count):
    # Split data to 60% training 20% validation and 20% testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    model = model1(X_train, X_val, y_train, y_val)
    y_pred = model.predict(X_test)

    print('Model:', i+1)
    y_pred = np.round(y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print('Mean Squared Error:', mse)
    print("--------------------")
    avg_mse += mse

print('Average Mean Squared Error:', avg_mse/count)

Epoch 1/20
563/563 [==============================] - 1s 1ms/step - loss: 12386.1943 - val_loss: 3169.8613
Epoch 2/20
563/563 [==============================] - 1s 989us/step - loss: 3224.3071 - val_loss: 3002.4209
Epoch 3/20
563/563 [==============================] - 1s 1000us/step - loss: 3121.0959 - val_loss: 3019.6687
Epoch 4/20
563/563 [==============================] - 1s 958us/step - loss: 3042.1355 - val_loss: 2909.5486
Epoch 5/20
563/563 [==============================] - 1s 933us/step - loss: 2996.9778 - val_loss: 2874.4570
Epoch 6/20
563/563 [==============================] - 1s 942us/step - loss: 2964.1426 - val_loss: 2827.1260
Epoch 7/20
563/563 [==============================] - 1s 931us/step - loss: 2923.3037 - val_loss: 2841.1707
Epoch 8/20
563/563 [==============================] - 1s 929us/step - loss: 2882.3889 - val_loss: 2938.6287
Epoch 9/20
563/563 [==============================] - 1s 928us/step - loss: 2852.2292 - val_loss: 2818.6968
Epoch 10/20
563/563 [=======

In [21]:
# Model-2: Add a fully connected layer with 32 neurons with sigmoid activation
# and glorot uniform kernel initializer. Add a fully connected layer layer with 8
# neurons, sigmoid activation and glorot normal as kernel initializer. Add a fully
# connected layer with 1 neuron, relu activation function and he uniform as
# kernel initializer. Use Adam optimizer with batch size 8, learning rate 0.01
# and epochs set to 20. Extract the features from second last fully connected
# layer (having 8 neurons) and model it using a Support Vector regressor.

from keras.models import Model
from sklearn.svm import SVR

def model2(X_train, X_val, y_train, y_val):
    model = Sequential()
    model.add(Dense(32, activation='sigmoid', kernel_initializer='glorot_uniform'))
    model.add(Dense(8, activation='sigmoid', kernel_initializer='glorot_normal'))
    model.add(Dense(1, activation='relu', kernel_initializer='he_uniform'))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

    model.fit(X_train, y_train, batch_size=8, epochs=20, validation_data=(X_val, y_val))

    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-2].output)
    intermediate_output = intermediate_layer_model.predict(X_train)

    svr = SVR()
    svr.fit(intermediate_output, y_train)

    return model, svr

count = 5
avg_mse = 0

for i in range(count):
    # Split data to 60% training 20% validation and 20% testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    model, svr = model2(X_train, X_val, y_train, y_val)
    
    intermediate_output = model.layers[-2].output
    intermediate_layer_model = Model(inputs=model.input, outputs=intermediate_output)
    intermediate_output = intermediate_layer_model.predict(X_test)

    y_pred = svr.predict(intermediate_output)

    print('Model:', i+1)
    y_pred = np.round(y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print('Mean Squared Error:', mse)
    print("--------------------")
    avg_mse += mse

print('Average Mean Squared Error:', avg_mse/count)

Epoch 1/20
1125/1125 [==============================] - 2s 996us/step - loss: 88622.5859 - val_loss: 87336.4844
Epoch 2/20
1125/1125 [==============================] - 1s 958us/step - loss: 88622.5703 - val_loss: 87336.4844
Epoch 3/20
1125/1125 [==============================] - 1s 940us/step - loss: 88622.5469 - val_loss: 87336.4844
Epoch 4/20
1125/1125 [==============================] - 1s 921us/step - loss: 88622.5781 - val_loss: 87336.4844
Epoch 5/20
1125/1125 [==============================] - 1s 945us/step - loss: 88622.5859 - val_loss: 87336.4844
Epoch 6/20
1125/1125 [==============================] - 1s 935us/step - loss: 88622.5391 - val_loss: 87336.4844
Epoch 7/20
1125/1125 [==============================] - 1s 989us/step - loss: 88622.5547 - val_loss: 87336.4844
Epoch 8/20
1125/1125 [==============================] - 1s 964us/step - loss: 88622.6016 - val_loss: 87336.4844
Epoch 9/20
1125/1125 [==============================] - 1s 961us/step - loss: 88622.5391 - val_loss: 873

In [22]:
# Model-3: Extract the deep features from Model-I (from 2nd layer) and Model-
# 2 (from 2nd layer) stack the features horizontally and model it using a Support
# Vector Regressor.

def model3(X_train, X_val, y_train, y_val):
    model_1 = model1(X_train, X_val, y_train, y_val)
    model_2, _ = model2(X_train, X_val, y_train, y_val)

    intermediate_output1 = model_1.layers[-2].output
    intermediate_output2 = model_2.layers[-2].output

    intermediate_layer_model_1 = Model(inputs=model_1.input, outputs=intermediate_output1)

    intermediate_output1 = intermediate_layer_model_1.predict(X_train)

    intermediate_layer_model_2 = Model(inputs=model_2.input, outputs=intermediate_output2)

    intermediate_output2 = intermediate_layer_model_2.predict(X_train)

    intermediate_output = np.hstack((intermediate_output1, intermediate_output2))

    svr = SVR()
    svr.fit(intermediate_output, y_train)

    return model_1, model_2, svr

count = 5
avg_mse = 0

for i in range(count):
    # Split data to 60% training 20% validation and 20% testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    model_1, model_2, svr = model3(X_train, X_val, y_train, y_val)

    intermediate_output1 = model_1.layers[-2].output
    intermediate_output2 = model_2.layers[-2].output

    intermediate_layer_model_1 = Model(inputs=model_1.input, outputs=intermediate_output1)

    intermediate_output1 = intermediate_layer_model_1.predict(X_test)

    intermediate_layer_model_2 = Model(inputs=model_2.input, outputs=intermediate_output2)

    intermediate_output2 = intermediate_layer_model_2.predict(X_test)

    intermediate_output = np.hstack((intermediate_output1, intermediate_output2))

    y_pred = svr.predict(intermediate_output)

    print('Model:', i+1)
    y_pred = np.round(y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print('Mean Squared Error:', mse)
    print("--------------------")
    avg_mse += mse

print('Average Mean Squared Error:', avg_mse/count)

    

Epoch 1/20
563/563 [==============================] - 1s 1ms/step - loss: 11945.1299 - val_loss: 3144.1050
Epoch 2/20
563/563 [==============================] - 1s 948us/step - loss: 3207.5786 - val_loss: 3050.7461
Epoch 3/20
563/563 [==============================] - 1s 944us/step - loss: 3104.2451 - val_loss: 2954.7461
Epoch 4/20
563/563 [==============================] - 1s 954us/step - loss: 3062.3867 - val_loss: 3009.6094
Epoch 5/20
563/563 [==============================] - 1s 957us/step - loss: 3024.0359 - val_loss: 2870.3667
Epoch 6/20
563/563 [==============================] - 1s 939us/step - loss: 2983.6787 - val_loss: 2916.1250
Epoch 7/20
563/563 [==============================] - 1s 986us/step - loss: 2919.1902 - val_loss: 2819.6453
Epoch 8/20
563/563 [==============================] - 1s 993us/step - loss: 2906.6006 - val_loss: 2790.2700
Epoch 9/20
563/563 [==============================] - 1s 936us/step - loss: 2873.9065 - val_loss: 2809.4346
Epoch 10/20
563/563 [========

In [23]:
# Model-4: Extract the deep features from Model-I and Model-2 stack the
# features horizontally, reduce the dimension to either 8, IO or 12 using
# principal component analysis (PCA) and model the reduced features using a
# Random Forest classifier. Identify the best number of reduced components of
# PCA.

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

def model4(X_train, X_val, y_train, y_val):
    model_1 = model1(X_train, X_val, y_train, y_val)
    model_2, _ = model2(X_train, X_val, y_train, y_val)

    intermediate_output1 = model_1.layers[-2].output
    intermediate_output2 = model_2.layers[-2].output

    intermediate_layer_model_1 = Model(inputs=model_1.input, outputs=intermediate_output1)

    intermediate_output1 = intermediate_layer_model_1.predict(X_train)

    intermediate_layer_model_2 = Model(inputs=model_2.input, outputs=intermediate_output2)

    intermediate_output2 = intermediate_layer_model_2.predict(X_train)

    intermediate_output = np.hstack((intermediate_output1, intermediate_output2))

    pca = PCA(n_components=8)
    pca.fit(intermediate_output)
    intermediate_output = pca.transform(intermediate_output)

    rf = RandomForestRegressor()
    rf.fit(intermediate_output, y_train)

    return model_1, model_2, pca, rf

count = 5
avg_mse = 0

for i in range(count):
    # Split data to 60% training 20% validation and 20% testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    model_1, model_2, pca, rf = model4(X_train, X_val, y_train, y_val)

    intermediate_output1 = model_1.layers[-2].output
    intermediate_output2 = model_2.layers[-2].output

    intermediate_layer_model_1 = Model(inputs=model_1.input, outputs=intermediate_output1)
    intermediate_output1 = intermediate_layer_model_1.predict(X_test)

    intermediate_layer_model_2 = Model(inputs=model_2.input, outputs=intermediate_output2)
    intermediate_output2 = intermediate_layer_model_2.predict(X_test)

    intermediate_output = np.hstack((intermediate_output1, intermediate_output2))

    intermediate_output = pca.transform(intermediate_output)

    y_pred = rf.predict(intermediate_output)

    print('Model:', i+1)
    y_pred = np.round(y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print('Mean Squared Error:', mse)
    print("--------------------")

    avg_mse += mse

print('Average Mean Squared Error:', avg_mse/count)

Epoch 1/20
563/563 [==============================] - 1s 1ms/step - loss: 12018.9316 - val_loss: 3243.3008
Epoch 2/20
563/563 [==============================] - 1s 952us/step - loss: 3221.0396 - val_loss: 3031.9114
Epoch 3/20
563/563 [==============================] - 1s 952us/step - loss: 3121.9602 - val_loss: 2995.2710
Epoch 4/20
563/563 [==============================] - 1s 950us/step - loss: 3065.3274 - val_loss: 2928.5117
Epoch 5/20
563/563 [==============================] - 1s 1ms/step - loss: 3002.2068 - val_loss: 2973.2566
Epoch 6/20
563/563 [==============================] - 1s 970us/step - loss: 2976.8367 - val_loss: 2833.1746
Epoch 7/20
563/563 [==============================] - 1s 1ms/step - loss: 2941.4402 - val_loss: 2808.6130
Epoch 8/20
563/563 [==============================] - 1s 961us/step - loss: 2891.0303 - val_loss: 2784.0286
Epoch 9/20
563/563 [==============================] - 1s 1ms/step - loss: 2846.9150 - val_loss: 2838.1917
Epoch 10/20
563/563 [==============

QNo 5

For model1: Average Mean Squared Error: 2823.2824

For model2: Average Mean Squared Error: 3557.7468

For model3: Average Mean Squared Error: 2942.814533333333

For model4: Average Mean Squared Error: 2717.3895333333335

so by seeing this avg mean square error we can conclude model 4 is best but when we also consider time to train model 1 is faster